### LABELED DATA

In [1]:
from utils.models import DataItem
lines =open('/home/ec2-user/SageMaker/mariano/notebooks/05. High Recall V2/labeled_data.csv','r').read().splitlines()[1:]
lines = [line.split(';') for line in lines ]
lines[:3]
labeled_data = [DataItem(id_) for id_,_ in lines]

idx = 0
for id_,label in lines:
        item = labeled_data[idx]
        idx+=1
        assert item.id_ == id_
        if label == 'R':
            item.set_relevant()
        else:
            item.set_irrelevant()
            assert label=='I'


lines = open('./auxiliary_notebooks/in_canada.csv', 'r').read().splitlines()
irrelevants = [DataItem(line.split(';')[0]) for line in lines if line.split(';')[1]=='False']
for item in irrelevants:
    item.set_irrelevant()
labeled_data = labeled_data + irrelevants
print(f'{len(labeled_data):,}')

114,492


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
params = {'C':[0.5, 1, 5, 7, 15, 50]}
GS = GridSearchCV(
                  SVC(kernel='poly'),
                  params
                  ,scoring=['f1','accuracy','precision', 'recall'], 
                  cv=5,
                  verbose=4, 
                  refit='f1')    

X = DataItem.get_X(labeled_data, type_=DataItem.TYPE_GLOVE600)
y = DataItem.get_y(labeled_data)
rta = GS.fit(X,y)
df = (pd.DataFrame(rta.cv_results_)[["params","mean_test_accuracy", "mean_test_precision","mean_test_recall","mean_test_f1"]])
df.columns=['params','mean accuracy', 'mean precision','mean recall','mean f1',]
print(df)
info(f'Best Params: {df.best_params_}')
# clf_g600 = GS.best_estimator_

# del(X,y)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV 1/5] END C=0.5; accuracy: (test=0.994) f1: (test=0.496) precision: (test=0.424) recall: (test=0.598) total time= 1.1min
[CV 2/5] END C=0.5; accuracy: (test=0.998) f1: (test=0.682) precision: (test=1.000) recall: (test=0.517) total time= 1.2min
[CV 3/5] END C=0.5; accuracy: (test=0.997) f1: (test=0.628) precision: (test=0.964) recall: (test=0.466) total time= 1.2min
[CV 4/5] END C=0.5; accuracy: (test=0.998) f1: (test=0.674) precision: (test=1.000) recall: (test=0.509) total time= 1.2min
[CV 5/5] END C=0.5; accuracy: (test=0.998) f1: (test=0.718) precision: (test=1.000) recall: (test=0.560) total time= 1.2min
[CV 1/5] END C=1; accuracy: (test=0.992) f1: (test=0.485) precision: (test=0.371) recall: (test=0.701) total time= 1.6min
[CV 2/5] END C=1; accuracy: (test=0.998) f1: (test=0.746) precision: (test=1.000) recall: (test=0.595) total time= 1.5min
[CV 3/5] END C=1; accuracy: (test=0.998) f1: (test=0.692) precision: (test=0

In [2]:
from sklearn.svm import SVC
 

X = DataItem.get_X(labeled_data, type_=DataItem.TYPE_GLOVE600)
y = DataItem.get_y(labeled_data)

clf_g606 = SVC(kernel='poly',C=5, probability=True)

clf_g606.fit(X,y)

SVC(C=5, kernel='poly', probability=True)

In [3]:
import os
import pickle
import numpy as np

prediction_files = os.listdir('../04. Model of DP/predictions/')
suggestions = [DataItem(file_[:-5]) for file_ in  prediction_files]
print(suggestions[0])

DataItem(id=1323603426, source=GM1, label=U)


In [9]:

# prediction_files = os.listdir('../04. Model of DP/predictions/')
# suggestions = [DataItem(file_[:-5]) for file_ in  prediction_files]

X = DataItem.get_X(suggestions, type_=DataItem.TYPE_GLOVE600)

yhat = clf_g606.predict_proba(X)

idx=0
for item in suggestions:
    prediction_file = '../04. Model of DP/predictions/'+item.id_+'_v3.p'
    assert os.path.isfile(prediction_file)
    
    y = pickle.load(open(prediction_file,'rb'))
    
    assert len(y)==3 and y.shape==(3,) and np.sum(y[1:])==0
    y[1] = yhat[idx,1]
#     print(y)
    pickle.dump(y, open(prediction_file,'wb'))
    idx+=1


In [7]:
yhat[:5,:]

array([[0.96132924, 0.03867076],
       [0.85273564, 0.14726436],
       [0.02990042, 0.97009958],
       [0.2389065 , 0.7610935 ],
       [0.16245994, 0.83754006]])

In [ ]:
# prediction_files = os.listdir('../04. Model of DP/predictions/')
# suggestions = [DataItem(file_[:-5]) for file_ in  prediction_files]

# X = DataItem.get_X(suggestions, type_=DataItem.TYPE_GLOVE300)

# yhat = clf_g300.predict_proba(X)

# idx=0
# for item in suggestions:
    